<a href="https://colab.research.google.com/github/erberry/ThinkML/blob/main/qwen_7b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers==4.31.0 accelerate tiktoken einops transformers_stream_generator

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 41.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 40.1 MB/s eta 0:00:00
  Created wheel for transformers_stream_generator: filename=transformers_stream_generator-0.0.4-py3-none-any.whl size=12319 sha256=4a2966972cbd2895791cbd91c035704739f33a88d8a93ccf76ec572cea253d70
  Stored in directory: /root/.cache/pip/wheels/47/1d/3c/92d88493ed40c0d9be60a391eb76c9a56e9f9b7542cb789401
Successfully built transformers_stream_generator


In [2]:
!pip install -i https://test.pypi.org/simple/ bitsandbytes

Looking in indexes: https://test.pypi.org/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.8/95.8 MB 8.8 MB/s eta 0:00:00


In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from transformers.generation import GenerationConfig

# Note: The default behavior now has injection attack prevention off.
# To remove the strategy, you can add `allowed_special`, which accepts the string "all" or a `set` of special tokens.
# For example: tokens = tokenizer(text, allowed_special="all")
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen-7B-Chat", trust_remote_code=True)

# use bf16
# model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen-7B-Chat", device_map="auto", trust_remote_code=True, bf16=True).eval()
# use fp16
# model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen-7B-Chat", device_map="auto", trust_remote_code=True, fp16=True).eval()
# use cpu only
# model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen-7B-Chat", device_map="cpu", trust_remote_code=True).eval()
# use auto mode, automatically select precision based on the device.
# model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen-7B-Chat", device_map="auto", trust_remote_code=True).eval()



# Specify hyperparameters for generation
# model.generation_config = GenerationConfig.from_pretrained("Qwen/Qwen-7B-Chat", trust_remote_code=True, quantization_config=quantization_config) # 可指定不同的生成长度、top_p等相关超参

# quantization configuration for Int8 (8 bits)
quantization_config = BitsAndBytesConfig(load_in_8bit=True)
model = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen-7B-Chat",
    quantization_config=quantization_config,
    trust_remote_code=True,
).eval()

# 第一轮对话 1st dialogue turn
response, history = model.chat(tokenizer, "你好", history=None)
print(response)
# 你好！很高兴为你提供帮助。

# 第二轮对话 2nd dialogue turn
response, history = model.chat(tokenizer, "给我讲一个年轻人奋斗创业最终取得成功的故事。", history=history)
print(response)
# 这是一个关于一个年轻人奋斗创业最终取得成功的故事。
# 故事的主人公叫李明，他来自一个普通的家庭，父母都是普通的工人。从小，李明就立下了一个目标：要成为一名成功的企业家。
# 为了实现这个目标，李明勤奋学习，考上了大学。在大学期间，他积极参加各种创业比赛，获得了不少奖项。他还利用课余时间去实习，积累了宝贵的经验。
# 毕业后，李明决定开始自己的创业之路。他开始寻找投资机会，但多次都被拒绝了。然而，他并没有放弃。他继续努力，不断改进自己的创业计划，并寻找新的投资机会。
# 最终，李明成功地获得了一笔投资，开始了自己的创业之路。他成立了一家科技公司，专注于开发新型软件。在他的领导下，公司迅速发展起来，成为了一家成功的科技企业。
# 李明的成功并不是偶然的。他勤奋、坚韧、勇于冒险，不断学习和改进自己。他的成功也证明了，只要努力奋斗，任何人都有可能取得成功。

# 第三轮对话 3rd dialogue turn
response, history = model.chat(tokenizer, "给这个故事起一个标题", history=history)
print(response)
# 《奋斗创业：一个年轻人的成功之路》


A new version of the following files was downloaded from https://huggingface.co/Qwen/Qwen-7B-Chat:
- tokenization_qwen.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/Qwen/Qwen-7B-Chat:
- configuration_qwen.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:147: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:147: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:147: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('http'), PosixPath('8013'), PosixPath('//172.28.0.1')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:147: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//colab.research.google.com/tun/m/cc483011

A new version of the following files was downloaded from https://huggingface.co/Qwen/Qwen-7B-Chat:
- qwen_generation_utils.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/Qwen/Qwen-7B-Chat:
- modeling_qwen.py
- qwen_generation_utils.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

你好！很高兴见到你。
这是一个关于一个年轻人如何在艰苦的努力和奋斗中取得成功的故事。

这个年轻人名叫李明，他一直有一个梦想：成为一名成功的企业家。他的父母也一直鼓励他去实现自己的梦想。

李明开始自学商业知识，并寻找机会实践。他曾经在一家小公司担任过销售经理，但是他的心中一直有一个更大的梦想。

于是，李明决定开始自己的创业之旅。他开始进行市场调研，寻找商机。在经过多次尝试和失败之后，他终于找到了一个可行的商业模式。

李明开始积极筹备自己的公司，并积极寻找投资者。他花费了大量的时间和精力来策划和实施他的计划，并不断努力改善他的产品和服务。

最终，李明的公司获得了成功，并成为行业内的领导者。他的梦想也终于实现了。他的成功故事鼓舞了许多年轻人，让他们相信只要努力奋斗，就一定能够实现自己的梦想。
《梦想成真：一个年轻人的奋斗历程》


In [5]:
response, _ = model.chat(tokenizer, "描述蓝色", history=None)
print(response)

蓝色是一种常见的颜色，它通常被描述为一种冷色调，是天空和水的颜色。蓝色在可见光谱的蓝色部分，波长范围约为460-495纳米。在心理学上，蓝色常常被与平静、安逸和放松联系在一起，也与信任和忠诚有关。蓝色在商业和文化中也具有多种含义，例如在服装和室内设计中使用蓝色可以传达专业和稳重的形象。


In [6]:
response, _ = model.chat(tokenizer, "翻译这句话 “你好，请问天安门怎么走？“ 到英文", history=None)
print(response)
response, _ = model.chat(tokenizer, "写一首关于爱得诗", history=None)
print(response)
response, _ = model.chat(tokenizer, "解释重力得概念", history=None)
print(response)
response, _ = model.chat(tokenizer, "蒙娜丽莎是谁画的", history=None)
print(response)
response, _ = model.chat(tokenizer, "对64开方", history=None)
print(response)
response, _ = model.chat(tokenizer, "64的平方根", history=None)
print(response)
response, _ = model.chat(tokenizer, "你能唱一首《我爱你中国》吗", history=None)
print(response)
response, _ = model.chat(tokenizer, "详述一下光合作用的过程", history=None)
print(response)
response, _ = model.chat(tokenizer, "生命的意义是什么？", history=None)
print(response)
response, _ = model.chat(tokenizer, "'如果所有的猫都是动物,而且一些动物是狗',这些陈述中得出什么结论？", history=None)
print(response)

Hello, how can I get to Tian'anmen Gate?
爱

爱是一种强烈的情感，  
它可以让我们燃烧，也可以让我们冷却。  
它可以让我们飞翔，也可以让我们坠落。  

爱是一种温柔的力量，  
它可以让我们温柔，也可以让我们坚强。  
它可以让我们相互支持，也可以让我们独立。  

爱是一种无私的情感，  
它可以让我们付出，也可以让我们收获。  
它可以让我们彼此理解，也可以让我们彼此分离。  

爱是一种永恒的主题，  
它可以让我们珍惜，也可以让我们遗忘。  
它可以让我们一起成长，也可以让我们一起消亡。
重力是一种自然现象，它是物体之间的相互吸引力。重力是由物体的质量产生的，物体的质量越大，其引力也就越强。重力的方向是垂直于物体表面，指向地球的中心。重力是宇宙中最重要的力量之一，它使得物体能够保持其运动状态，也是物体之间的相互作用力之一。
蒙娜丽莎是由意大利文艺复兴时期的画家列奥纳多·达·芬奇（Leonardo da Vinci）所画。这幅画作是他的杰作之一，被誉为世界最伟大的艺术作品之一。
64的平方根是8，所以64的开方根是8。
64的平方根是8。
对不起，我无法唱歌。我只能回答问题和提供信息。
光合作用是植物和其他一些生物利用太阳能来合成有机物质的过程。它的过程分为两个阶段：光反应和暗反应。

光反应发生在叶绿体的膜上，需要太阳光作为能源。在光反应中，光能被转化成化学能，其中最重要的是ATP（三磷酸腺苷）和NADPH（辅酶NADPH）。这些化学能用于暗反应。

暗反应发生在叶绿体的基质中，需要ATP和NADPH作为能源。在暗反应中，植物将二氧化碳和水转化成糖类，这些糖类可以被植物用来生长和繁殖。同时，暗反应也会产生氧气，这是光合作用的最终产物。

光合作用是一个非常复杂的过程，它涉及到许多不同的生物分子和化学反应。然而，以上是光合作用的基本过程。
生命的意义是复杂而多样的，不同的人可能会有不同的看法。从生物学家的角度来看，生命的意义在于维持生命的存在，包括繁衍后代、保护自己和适应环境。从哲学的角度来看，生命的意义可能在于实现个人的价值观和目标，或者在于探索和理解世界。从宗教的角度来看，生命的意义可能在于追求更高的精神境界或者与神的联系。
根据提供的信息，可以得出结论：有些动物是猫和狗。
